https://play.beam.apache.org/

In [ ]:
!pip install -r requirements.txt

In [35]:
import apache_beam as beam

import pandas as pd

In [2]:

from IPython.display import clear_output

In [3]:
# pip install pandas==1.5.2 apache-beam==2.48.0

# Map , combiners, GroupBy

In [5]:
!rm example-*
with beam.Pipeline() as p:

  (p | beam.Create(range(0, 11,2)[1:])
     | beam.combiners.Mean.Globally()
     | beam.io.textio.WriteToText('example-output')
    )
clear_output()
!cat example-output*

6.0


In [8]:
!rm example-*


with beam.Pipeline() as p:

  (p | beam.Create(range(1, 11))
     | beam.Filter(lambda num: num % 2 == 0)
     | 'Write' >> beam.io.textio.WriteToText('example-output'))
    
clear_output()
!cat example-output*

2
4
6
8
10


In [9]:
!rm example-*

with beam.Pipeline() as p:

  (p | beam.Create(range(1, 11))
     | beam.Map(lambda num: num * 2)
     | 'Write' >> beam.io.textio.WriteToText('example-output'))
    
clear_output()
!cat example-output*

2
4
6
8
10
12
14
16
18
20


In [11]:
!rm example-*
class FilterOutEvenNumber(beam.DoFn):

    def process(self, element):
        if element % 2 == 0:
            yield element


with beam.Pipeline() as p:
  (p | beam.Create(range(1, 11))
     | beam.ParDo(FilterOutEvenNumber())
    | 'Write' >> beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

2
4
6
8
10


In [12]:
import numpy as np

In [13]:
list(range(0, 11,2)),np.mean(list(range(0, 11,2)))

([0, 2, 4, 6, 8, 10], 5.0)

In [14]:

!rm example-*
with beam.Pipeline() as p:

  (p | beam.Create(range(1, 11))
     | beam.combiners.Top.Largest(4)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[10, 9, 8, 7]


In [15]:
!rm example-*
with beam.Pipeline() as p:

  (p | beam.Create(range(1, 11))
     | beam.combiners.Top.Smallest(2)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[1, 2]


In [22]:
!rm example-*
with beam.Pipeline() as p:

  (p | beam.Create(range(1, 11))
     | beam.CombineGlobally(min)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

1


In [23]:
%%writefile inputtext.txt
10
20
30
40
100

Overwriting inputtext.txt


In [24]:

!rm example-*

with beam.Pipeline() as p:
    # | beam.Create([10, 20, 30, 40, 50])
  (p | beam.io.ReadFromText('inputtext.txt')
     | beam.Map(lambda num: 2*int(num) + 5)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

25
45
65
85
205


In [16]:
%%writefile inputtext2.txt
10,20,30
20,30,35
30,35,40
40,45,50
100,105,110

Overwriting inputtext2.txt


In [25]:

!rm example-*

with beam.Pipeline() as p:
    # | beam.Create([10, 20, 30, 40, 50])
  (p | beam.io.ReadFromText('inputtext2.txt')
     | beam.Map(lambda line: line.split(","))
     | beam.Map(lambda nums: [2*int(num) + 5 for num in nums])
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[25, 45, 65]
[45, 65, 75]
[65, 75, 85]
[85, 95, 105]
[205, 215, 225]


In [26]:
%%writefile inputtext3.txt
10,20,30,B,C
20,30,35,A,D
30,35,40,K,E
40,45,50,N,C
100,105,110,M,C

Overwriting inputtext3.txt


In [27]:
def strint(line):
    col=[]
    for x in line:
        try:
            col.append(int(x))
        except:
            col.append(x)
    return col

In [40]:

!rm example-*
with beam.Pipeline() as p:
    # | beam.Create([10, 20, 30, 40, 50])
  (p | beam.io.ReadFromText('inputtext3.txt')
     | beam.Map(lambda line : line.split(","))
     | beam.Map(lambda num : strint(num))
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[10, 20, 30, 'B', 'C']
[20, 30, 35, 'A', 'D']
[30, 35, 40, 'K', 'E']
[40, 45, 50, 'N', 'C']
[100, 105, 110, 'M', 'C']


In [31]:
def count_ones(word_ones):
    (word, ones) = word_ones
    return (word, sum(ones))

In [76]:
%%writefile inputtext2.txt
10,20,30,C
20,30,35,D
30,35,40,E
40,45,50,C
100,105,110,C
100,105,110,A
100,105,110,A
100,105,110,A
100,105,110,D
100,105,110,D
100,105,110,D
100,105,110,D
100,105,110,D
100,105,110,D
10,20,30,F
20,30,35,F
30,35,40,F
40,45,50,F
30,35,40,Z
40,45,50,Z
100,105,110,Z
100,105,110,Z

Overwriting inputtext2.txt


In [66]:
import numpy as np

In [90]:
# inputtext2.txt
# 10,20,30,C
# 20,30,35,D
# 30,35,40,E
# 40,45,50,C
# 100,105,110,C
# 100,105,110,A
# 100,105,110,A
# 100,105,110,A
# 100,105,110,D
# 100,105,110,D
# 100,105,110,D
# 100,105,110,D
# 100,105,110,D
# 100,105,110,D
!rm example-*

with beam.Pipeline() as p:
    (p | beam.io.ReadFromText('inputtext2.txt')
       | beam.Map(lambda line : line.split(","))
       | beam.Map(lambda fields : (fields[3] ,[int(fields[0]),int(fields[1]),int(fields[2])]))
       | beam.GroupByKey()
       | beam.Map(lambda fields : (fields[0],{"sum":np.sum(fields[1]),"mean":np.mean(fields[1]),"max":np.max(fields[1]),"min":np.min(fields[1]),
                                             "std":np.std(fields[1]),"ptp":np.ptp(fields[1])}))
       | beam.io.textio.WriteToText('example-output'))


clear_output()
!cat example-output*

('C', {'sum': 510, 'mean': 56.666666666666664, 'max': 110, 'min': 10, 'std': 36.132472314464664, 'ptp': 100})
('D', {'sum': 1975, 'mean': 94.04761904761905, 'max': 110, 'min': 20, 'std': 27.195020886142604, 'ptp': 90})
('E', {'sum': 105, 'mean': 35.0, 'max': 40, 'min': 30, 'std': 4.08248290463863, 'ptp': 10})
('A', {'sum': 945, 'mean': 105.0, 'max': 110, 'min': 100, 'std': 4.08248290463863, 'ptp': 10})
('F', {'sum': 385, 'mean': 32.083333333333336, 'max': 50, 'min': 10, 'std': 10.889278621143315, 'ptp': 40})
('Z', {'sum': 870, 'mean': 72.5, 'max': 110, 'min': 30, 'std': 32.94566233461799, 'ptp': 80})


In [95]:
np.sum([[10, 20, 30], [40, 45, 50], [100, 105, 110]])

510

In [96]:
import datetime
import time

In [93]:
datetime.datetime.fromtimestamp(time.time())

datetime.datetime(2023, 7, 14, 18, 0, 15, 74520)

In [94]:
datetime.datetime.strptime("22-12-21 17:57:43","%y-%m-%d %H:%M:%S")

datetime.datetime(2022, 12, 21, 17, 57, 43)

In [24]:
%%writefile inputtext4.txt
10,20,30,B,C,22-12-21 17:57:43
20,30,35,A,D,22-12-21 17:57:43
30,35,40,K,E,22-12-21 17:57:43
40,45,50,N,C,22-12-21 17:57:43
100,105,110,M,C,22-12-21 17:57:43
10,20,30,B,C,22-12-21 17:57:43
20,30,35,A,D,22-12-21 17:57:43
30,35,40,K,E,22-12-21 17:57:43
40,45,50,N,C,22-12-21 17:57:43
100,105,110,M,C,22-12-21 17:57:43
10,20,30,B,C,22-12-21 17:57:43
20,30,35,A,D,22-12-21 17:57:43
30,35,40,K,E,22-12-21 17:57:43
40,45,50,N,C,22-12-21 17:57:43
100,105,110,M,C,22-12-21 17:57:43
10,20,30,B,C,22-12-21 17:57:43
20,30,35,A,D,22-12-21 17:57:43
30,35,40,K,E,22-12-21 17:57:43
40,45,50,N,C,22-12-21 17:57:43
100,105,110,M,C,22-12-21 17:57:43


Overwriting inputtext4.txt


In [25]:
def strintdate(line):
    col=[]
    for x in line:
        try:
            try:
                try:
                    col.append(int(x))
                except:
                    col.append(datetime.datetime.strptime(x,"%y-%m-%d %H:%M:%S"))
            except:
                 col.append(x)
        except:pass
    return col

In [26]:
def stats(line):
    col=[]
    sums=0
    for x in line:
        col.append(x)
    arr=[]
    for i,x in enumerate(line):
        if i<=2:
            arr.append(int(x))
        else:
            pass
    col.append(np.round(np.mean(arr)))    
    col.append(np.round(np.std(arr)))
    col.append(np.round(np.var(arr))) 
    col.append(np.round(np.min(arr)))    
    col.append(np.round(np.max(arr)))    
    col.append(np.round(np.ptp(arr)))      
    return col

In [27]:


with beam.Pipeline() as p:
    # | beam.Create([10, 20, 30, 40, 50])
  (p | beam.io.ReadFromText('inputtext4.txt')
     | beam.Map(lambda line : line.split(","))
     | beam.Map(lambda line : strintdate(line))
     | beam.Map(lambda line : stats(line))
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[10, 20, 30, 'B', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 20.0, 8.0, 67.0, 10, 30, 20]
[20, 30, 35, 'A', 'D', datetime.datetime(2022, 12, 21, 17, 57, 43), 28.0, 6.0, 39.0, 20, 35, 15]
[30, 35, 40, 'K', 'E', datetime.datetime(2022, 12, 21, 17, 57, 43), 35.0, 4.0, 17.0, 30, 40, 10]
[40, 45, 50, 'N', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 45.0, 4.0, 17.0, 40, 50, 10]
[100, 105, 110, 'M', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 105.0, 4.0, 17.0, 100, 110, 10]
[10, 20, 30, 'B', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 20.0, 8.0, 67.0, 10, 30, 20]
[20, 30, 35, 'A', 'D', datetime.datetime(2022, 12, 21, 17, 57, 43), 28.0, 6.0, 39.0, 20, 35, 15]
[30, 35, 40, 'K', 'E', datetime.datetime(2022, 12, 21, 17, 57, 43), 35.0, 4.0, 17.0, 30, 40, 10]
[40, 45, 50, 'N', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 45.0, 4.0, 17.0, 40, 50, 10]
[100, 105, 110, 'M', 'C', datetime.datetime(2022, 12, 21, 17, 57, 43), 105.0, 4.0, 17.0, 100, 110, 10]
[10, 20, 30, 'B', 

In [29]:
#CoGroupByKey

class WordsAlphabet:

    def __init__(self, alphabet, fruit, country,nationality):
        self.alphabet = alphabet
        self.fruit = fruit
        self.country = country
        self.nationality = nationality

    def __str__(self):
        return "WordsAlphabet(alphabet:'%s', fruit='%s', country='%s', nationality='%s')" % (self.alphabet, self.fruit, self.country,self.nationality)


def apply_transforms(fruits, countries,nationalities):
    def map_to_alphabet_kv(word):
        return (word[0], word)

    def cogbk_result_to_wordsalphabet(cgbk_result):
        (alphabet, words) = cgbk_result
        return WordsAlphabet(alphabet, words['fruits'][0], words['countries'][0],words['nationalities'][0])

    fruits_kv = (fruits | 'Fruit to KV' >> beam.Map(map_to_alphabet_kv))
    countries_kv = (countries | 'Country to KV' >> beam.Map(map_to_alphabet_kv))
    nationalities_kv = (nationalities | 'nationality to KV' >> beam.Map(map_to_alphabet_kv))
    return ({'fruits': fruits_kv, 'countries': countries_kv,'nationalities': nationalities_kv}
            | beam.CoGroupByKey()
            | beam.Map(cogbk_result_to_wordsalphabet))


with beam.Pipeline() as p:

    fruits = p | 'Fruits' >> beam.Create(['apple', 'banana', 'cherry','doctor'])
    countries = p | 'Countries' >> beam.Create(['australia', 'brazil', 'canada','dutchland'])
    nationalities = p | 'Nationalities' >> beam.Create(['australian', 'brazilian', 'canadian','dutch'])

    (apply_transforms(fruits, countries,nationalities)
    | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

WordsAlphabet(alphabet:'a', fruit='apple', country='australia', nationality='australian')
WordsAlphabet(alphabet:'b', fruit='banana', country='brazil', nationality='brazilian')
WordsAlphabet(alphabet:'c', fruit='cherry', country='canada', nationality='canadian')
WordsAlphabet(alphabet:'d', fruit='doctor', country='dutchland', nationality='dutch')
10
20
30
40
50


In [31]:
import numpy as np

example_list=[10, 20, 50, 70, 90]


class AverageFn(beam.CombineFn):

    def create_accumulator(self):
        return 0.0, 0

    def add_input(self, accumulator, element):
        (sum, i) = accumulator
        # print(accumulator)
        # print(element)
        return sum + element, i + 1

    def merge_accumulators(self, accumulators):
        sums, counts = zip(*accumulators)
        print(accumulators)
        return sum(sums), sum(counts)

    def extract_output(self, accumulator):
        (sum, count) = accumulator
        return sum / count if count else float('NaN')


with beam.Pipeline() as p:
    clear_output()
    (p | beam.Create(example_list)
     | beam.CombineGlobally(AverageFn())
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*
np.mean(example_list)

48.0
10
20
30
40
50


48.0

In [30]:
with beam.Pipeline() as p:
  (p | beam.Create(range(1, 11))
     | beam.combiners.Mean.Globally()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

5.5
10
20
30
40
50


In [32]:
with beam.Pipeline() as p:
  (p | beam.Create(example_list)
     | beam.combiners.Mean.Globally()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

48.0
10
20
30
40
50


In [33]:
with beam.Pipeline() as p:
  (p | beam.Create(range(1, 11))
     | beam.combiners.Count.Globally()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

10
10
20
30
40
50


In [34]:
with beam.Pipeline() as p:
  (p | beam.Create(range(1, 11))
     | beam.combiners.Latest.Globally()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

10
10
20
30
40
50


In [37]:
with beam.Pipeline() as p:
  (p | beam.Create(range(1, 11))
     | beam.combiners.ToList()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
10
20
30
40
50


In [38]:
#   Licensed to the Apache Software Foundation (ASF) under one
#   or more contributor license agreements.  See the NOTICE file
#   distributed with this work for additional information
#   regarding copyright ownership.  The ASF licenses this file
#   to you under the Apache License, Version 2.0 (the
#   "License"); you may not use this file except in compliance
#   with the License.  You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import datetime
import pytz


from apache_beam.transforms import window




class Event:
    def __init__(self, id, event, timestamp):
        self.id = id
        self.event = event
        self.timestamp = timestamp

    def __str__(self) -> str:
        return f'Event({self.id}, {self.event}, {self.timestamp})'


class AddTimestampDoFn(beam.DoFn):

    def process(self, element, **kwargs):
        unix_timestamp = element.timestamp.timestamp()
        yield window.TimestampedValue(element, unix_timestamp)


with beam.Pipeline() as p:

  (p | beam.Create(
      [Event(str(x), f'book-order_{x}', datetime.datetime(2020, 3, x, x, 20-x, 0, 0, tzinfo=pytz.UTC)) \
       for x in range(1,19) ] 
  )
     | beam.ParDo(AddTimestampDoFn())
    | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*



Event(1, book-order_1, 2020-03-01 01:19:00+00:00)
Event(2, book-order_2, 2020-03-02 02:18:00+00:00)
Event(3, book-order_3, 2020-03-03 03:17:00+00:00)
Event(4, book-order_4, 2020-03-04 04:16:00+00:00)
Event(5, book-order_5, 2020-03-05 05:15:00+00:00)
Event(6, book-order_6, 2020-03-06 06:14:00+00:00)
Event(7, book-order_7, 2020-03-07 07:13:00+00:00)
Event(8, book-order_8, 2020-03-08 08:12:00+00:00)
Event(9, book-order_9, 2020-03-09 09:11:00+00:00)
Event(10, book-order_10, 2020-03-10 10:10:00+00:00)
Event(11, book-order_11, 2020-03-11 11:09:00+00:00)
Event(12, book-order_12, 2020-03-12 12:08:00+00:00)
Event(13, book-order_13, 2020-03-13 13:07:00+00:00)
Event(14, book-order_14, 2020-03-14 14:06:00+00:00)
Event(15, book-order_15, 2020-03-15 15:05:00+00:00)
Event(16, book-order_16, 2020-03-16 16:04:00+00:00)
Event(17, book-order_17, 2020-03-17 17:03:00+00:00)
Event(18, book-order_18, 2020-03-18 18:02:00+00:00)
10
20
30
40
50


In [48]:
# !wget https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture15b/sparklect/shakes/kinglear.txt

--2023-07-14 13:23:52--  https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture15b/sparklect/shakes/kinglear.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176339 (172K) [text/plain]
Saving to: ‘kinglear.txt’

kinglear.txt        100%[===================>] 172,21K  --.-KB/s    in 0,1s    

2023-07-14 13:23:52 (1,60 MB/s) - ‘kinglear.txt’ saved [176339/176339]



In [49]:
%%writefile  wordcount.py
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""A word-counting workflow."""

# pytype: skip-file

import argparse
import logging
import re


from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions


class WordExtractingDoFn(beam.DoFn):
  """Parse each line of input text into words."""
  def process(self, element):
    """Returns an iterator over the words of this element.

    The element is a line of text.  If the line is blank, note that, too.

    Args:
      element: the element being processed

    Returns:
      The processed element.
    """
    # return re.findall(r'[\w\']+', element, re.UNICODE)
    return element.split()
    


def run(argv=None, save_main_session=True):
  """Main entry point; defines and runs the wordcount pipeline."""
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--input',
      dest='input',
      default='kinglear.txt',
      
      help='Input file to process.')
  parser.add_argument(
      '--output',
      dest='output',
      required=True,
      help='Output file to write results to.')
  known_args, pipeline_args = parser.parse_known_args(argv)

  # We use the save_main_session option because one or more DoFn's in this
  # workflow rely on global context (e.g., a module imported at module level).
  pipeline_options = PipelineOptions(pipeline_args)
  pipeline_options.view_as(SetupOptions).save_main_session = save_main_session

  # The pipeline will be run on exiting the with block.
  with beam.Pipeline(options=pipeline_options) as p:

    # Read the text file[pattern] into a PCollection.
    lines = p | 'Read' >> ReadFromText(known_args.input)

    counts = (
        lines
        | 'Split' >> (beam.ParDo(WordExtractingDoFn()).with_output_types(str))
        | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
        | 'GroupAndSum' >> beam.CombinePerKey(sum))

    # Format the counts into a PCollection of strings.
    def format_result(word, count):
      return '%s: %d' % (word, count)

    output = counts | 'Format' >> beam.MapTuple(format_result)

    # Write the output using a "Write" transform that has side effects.
    # pylint: disable=expression-not-assigned
    output | 'Write' >> WriteToText(known_args.output)


if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  run()


Overwriting wordcount.py


In [51]:
!python3 wordcount.py --output wordcount_output
clear_output()
!head -n 5 wordcount_output*

==> wordcount_output-00000-of-00001 <==
﻿1606: 1
THE: 7
TRAGEDY: 1
OF: 16
KING: 1

==> wordcount_output2-00000-of-00001 <==
KING: 243
LEAR: 236
DRAMATIS: 1
PERSONAE: 1
king: 65


In [52]:
%%writefile wordcount_with_metric.py
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""A word-counting workflow."""

# pytype: skip-file

import argparse
import logging
import re


from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions


class WordExtractingDoFn(beam.DoFn):
  """Parse each line of input text into words."""
  def __init__(self):
    # TODO(BEAM-6158): Revert the workaround once we can pickle super() on py3.
    # super().__init__()
    beam.DoFn.__init__(self)
    self.words_counter = Metrics.counter(self.__class__, 'words')
    self.word_lengths_counter = Metrics.counter(self.__class__, 'word_lengths')
    self.word_lengths_dist = Metrics.distribution(
        self.__class__, 'word_len_dist')
    self.empty_line_counter = Metrics.counter(self.__class__, 'empty_lines')

  def process(self, element):
    """Returns an iterator over the words of this element.

    The element is a line of text.  If the line is blank, note that, too.

    Args:
      element: the element being processed

    Returns:
      The processed element.
    """
    text_line = element.strip()
    if not text_line:
      self.empty_line_counter.inc(1)
    words = re.findall(r'[\w\']+', text_line, re.UNICODE)
    for w in words:
      self.words_counter.inc()
      self.word_lengths_counter.inc(len(w))
      self.word_lengths_dist.update(len(w))
    return words


def main(argv=None, save_main_session=True):
  """Main entry point; defines and runs the wordcount pipeline."""
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--input',
      dest='input',
      default='kinglear.txt',
      help='Input file to process.')
  parser.add_argument(
      '--output',
      dest='output',
      required=True,
      help='Output file to write results to.')
  known_args, pipeline_args = parser.parse_known_args(argv)

  # We use the save_main_session option because one or more DoFn's in this
  # workflow rely on global context (e.g., a module imported at module level).
  pipeline_options = PipelineOptions(pipeline_args)
  pipeline_options.view_as(SetupOptions).save_main_session = save_main_session
  p = beam.Pipeline(options=pipeline_options)

  # Read the text file[pattern] into a PCollection.
  lines = p | 'read' >> ReadFromText(known_args.input)

  # Count the occurrences of each word.
  def count_ones(word_ones):
    (word, ones) = word_ones
    return (word, sum(ones))

  counts = (
      lines
      | 'split' >> (beam.ParDo(WordExtractingDoFn()).with_output_types(str))
      | 'pair_with_one' >> beam.Map(lambda x: (x, 1))
      | 'group' >> beam.GroupByKey()
      | 'count' >> beam.Map(count_ones))

  # Format the counts into a PCollection of strings.
  def format_result(word_count):
    (word, count) = word_count
    return '%s: %d' % (word, count)

  output = counts | 'format' >> beam.Map(format_result)

  # Write the output using a "Write" transform that has side effects.
  # pylint: disable=expression-not-assigned
  output | 'write' >> WriteToText(known_args.output)

  result = p.run()
  result.wait_until_finish()

  # Do not query metrics when creating a template which doesn't run
  if (not hasattr(result, 'has_job')  # direct runner
      or result.has_job):  # not just a template creation
    empty_lines_filter = MetricsFilter().with_name('empty_lines')
    query_result = result.metrics().query(empty_lines_filter)
    if query_result['counters']:
      empty_lines_counter = query_result['counters'][0]
      logging.info('number of empty lines: %d', empty_lines_counter.result)

    word_lengths_filter = MetricsFilter().with_name('word_len_dist')
    query_result = result.metrics().query(word_lengths_filter)
    if query_result['distributions']:
      word_lengths_dist = query_result['distributions'][0]
      logging.info('average word length: %d', word_lengths_dist.result.mean)


if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  main()


Overwriting wordcount_with_metric.py


In [53]:
!python3 wordcount_with_metric.py --output wordcount_output2
clear_output()
!head -n 5 wordcount_output2*

1606: 1
THE: 7
TRAGEDY: 1
OF: 16
KING: 1


In [54]:
%%writefile wordcounttest.py
# -*- coding: utf-8 -*-
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""Test for the wordcount example."""

# pytype: skip-file

import collections
import logging
import re
import tempfile
import unittest

import pytest

from apache_beam.examples import wordcount
from apache_beam.testing.util import open_shards


@pytest.mark.examples_postcommit
class WordCountTest(unittest.TestCase):

  SAMPLE_TEXT = (
      u'a b c a b a\nacento gráfico\nJuly 30, 2018\n\n aa bb cc aa bb aa')

  def create_temp_file(self, contents):
    with tempfile.NamedTemporaryFile(delete=False) as f:
      f.write(contents.encode('utf-8'))
      return f.name

  def test_basics(self):
    temp_path = self.create_temp_file(self.SAMPLE_TEXT)
    expected_words = collections.defaultdict(int)
    for word in re.findall(r'[\w\']+', self.SAMPLE_TEXT, re.UNICODE):
      expected_words[word] += 1
    wordcount.run(['--input=%s*' % temp_path, '--output=%s.result' % temp_path],
                  save_main_session=False)
    # Parse result file and compare.
    results = []
    with open_shards(temp_path + '.result-*-of-*') as result_file:
      for line in result_file:
        match = re.search(r'(\S+): ([0-9]+)', line)
        if match is not None:
          results.append((match.group(1), int(match.group(2))))
    self.assertEqual(sorted(results), sorted(expected_words.items()))


if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  unittest.main()


Overwriting wordcounttest.py


In [57]:
!python3 wordcounttest.py

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:root:Default Python SDK image for environment is apache/beam_python3.10_sdk:2.48.0
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7fcb1b594c10> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7fcb1b594d30> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function pack_combiners at 0x7fcb1b595240> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function lift_combiners at 0x7fcb1b5952d0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_sdf at 0x7fcb1b595480> ====================
INFO:apache_beam.runners.portabil

In [58]:


with beam.Pipeline() as p:

  (p | beam.Create(['apple', 'ball', 'car', 'bear', 'cheetah', 'ant'])
     | beam.Map(lambda word: (word[0], word))
     | beam.GroupByKey()
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*



('a', ['apple', 'ant'])
('b', ['ball', 'bear'])
('c', ['car', 'cheetah'])
10
20
30
40
50


In [59]:
#   Licensed to the Apache Software Foundation (ASF) under one
#   or more contributor license agreements.  See the NOTICE file
#   distributed with this work for additional information
#   regarding copyright ownership.  The ASF licenses this file
#   to you under the Apache License, Version 2.0 (the
#   "License"); you may not use this file except in compliance
#   with the License.  You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.



class WordsAlphabet:

    def __init__(self, alphabet, fruit, country):
        self.alphabet = alphabet
        self.fruit = fruit
        self.country = country

    def __str__(self):
        return "WordsAlphabet(alphabet:'%s', fruit='%s', country='%s')" % (self.alphabet, self.fruit, self.country)


def apply_transforms(fruits, countries):
    def map_to_alphabet_kv(word):
        return (word[0], word)

    def cogbk_result_to_wordsalphabet(cgbk_result):
        (alphabet, words) = cgbk_result
        return WordsAlphabet(alphabet, words['fruits'][0], words['countries'][0])

    fruits_kv = (fruits | 'Fruit to KV' >> beam.Map(map_to_alphabet_kv))
    countries_kv = (countries | 'Country to KV' >> beam.Map(map_to_alphabet_kv))

    return ({'fruits': fruits_kv, 'countries': countries_kv}
            | beam.CoGroupByKey()
            | beam.Map(cogbk_result_to_wordsalphabet))


with beam.Pipeline() as p:

  fruits = p | 'Fruits' >> beam.Create(['apple', 'banana', 'cherry'])
  countries = p | 'Countries' >> beam.Create(['australia', 'brazil', 'canada'])

  (apply_transforms(fruits, countries)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*



WordsAlphabet(alphabet:'a', fruit='apple', country='australia')
WordsAlphabet(alphabet:'b', fruit='banana', country='brazil')
WordsAlphabet(alphabet:'c', fruit='cherry', country='canada')
10
20
30
40
50


In [60]:
#combine per key


PLAYER_1 = 'Player 1'
PLAYER_2 = 'Player 2'
PLAYER_3 = 'Player 3'

with beam.Pipeline() as p:

  (p | beam.Create([(PLAYER_1, 15), (PLAYER_2, 10), (PLAYER_1, 100),
                    (PLAYER_3, 25), (PLAYER_2, 75)])
     | beam.CombinePerKey(sum)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

('Player 1', 115)
('Player 2', 85)
('Player 3', 25)
10
20
30
40
50


In [61]:
#combine simple function


def sum(numbers):
    total = 0

    for num in numbers:
        total += num

    return total


with beam.Pipeline() as p:

  (p | beam.Create([1, 2, 3, 4, 5])
     | beam.CombineGlobally(sum)
     | beam.io.textio.WriteToText('example-output'))
clear_output()
!cat example-output*

15
10
20
30
40
50


In [62]:
#side input


class Person:
    def __init__(self, name, city, country='',continent='',num1="",num2="",avg=""):
        self.name = name
        self.city = city
        self.country = country
        self.continent= continent
        self.num1=num1
        self.num2=num2
        self.avg=avg      

    def __str__(self):
        return 'Person[' + self.name + ',' + self.city + ',' + self.country + ',' + self.continent +',' + self.num1+',' + self.num2+','  +self.avg+']'


class EnrichCountryDoFn(beam.DoFn):

    def process(self, element, cities_to_countries):
        yield Person(element.name, element.city,
                     cities_to_countries[element.city])
        
class EnrichContinentDoFn(beam.DoFn):

    def process(self, element, cities_to_continents):
        yield Person(element.name, element.city,element.country,
                     cities_to_continents[element.city])
        
class NumDoFn(beam.DoFn):

    def process(self, element, name_to_num1):
        yield Person(element.name, element.city,element.country,element.continent,
                     name_to_num1[element.name])
        
class NumDoFn2(beam.DoFn):

    def process(self, element, name_to_num1):
        yield Person(element.name, element.city,element.country,element.continent,element.num1,
                     name_to_num2[element.name])

class AvgDoFn(beam.DoFn):

    def process(self, element):
        yield Person(element.name, element.city,element.country,element.continent,element.num1,element.num2,
                     str(round((int(element.num1)+int(element.num2))/2,2)))
        
with beam.Pipeline() as p:

    cities_to_countries = {
      'Beijing': 'China',
      'London': 'United Kingdom',
      'San Francisco': 'United States',
      'Singapore': 'Singapore',
      'Sydney': 'Australia'
    }

    cities_to_continents = {
      'Beijing': 'Asia',
      'London': 'Europe',
      'San Francisco': 'America',
      'Singapore': 'Asia',
      'Sydney': 'Australia'
    }
    
    name_to_num1 = {
      'Henry': "25",
      'Jane': "30",
      'Lee': "35",
      'John': "45",
      'Alfred': "50"
    }
    
    name_to_num2 = {
      'Henry': "50",
      'Jane': "45",
      'Lee': "35",
      'John': "30",
      'Alfred': "25"
    }
    
    persons = [
      Person('Henry', 'Singapore'),
      Person('Jane', 'San Francisco'),
      Person('Lee', 'Beijing'),
      Person('John', 'Sydney'),
      Person('Alfred', 'London')
    ]
    
    
    (p  | 'Create' >> beam.Create(persons)
        | 'Add-countries' >> beam.ParDo(EnrichCountryDoFn(), cities_to_countries)
        | 'Add-continents' >> beam.ParDo(EnrichContinentDoFn(), cities_to_continents)
        | 'Add-num' >> beam.ParDo(NumDoFn(), name_to_num1)
        | 'Add-num2' >> beam.ParDo(NumDoFn2(), name_to_num2)
        | 'avg' >> beam.ParDo(AvgDoFn())     
        | 'Write' >> beam.io.textio.WriteToText('example-output'))
    
clear_output()
!cat example-output*

Person[Henry,Singapore,Singapore,Asia,25,50,37.5]
Person[Jane,San Francisco,United States,America,30,45,37.5]
Person[Lee,Beijing,China,Asia,35,35,35.0]
Person[John,Sydney,Australia,Australia,45,30,37.5]
Person[Alfred,London,United Kingdom,Europe,50,25,37.5]
10
20
30
40
50


# Side input

In [97]:

!rm example-*
class Person:
    def __init__(self, name, city, country=''):
        self.name = name
        self.city = city
        self.country = country

    def __str__(self):
        return 'Person[' + self.name + ',' + self.city + ',' + self.country + ']'


class EnrichCountryDoFn(beam.DoFn):
    def process(self, element, cities_to_countries):
      yield Person(element.name, element.city, cities_to_countries[element.city])


with beam.Pipeline() as p:
  cities_to_countries = p | "Side input" >> beam.Create([('Beijing', 'China'),
                                                         ('London', 'United Kingdom'),
                                                         ('San Francisco', 'United States'),
                                                         ('Singapore', 'Singapore'),
                                                         ('Sydney', 'Australia')])

  persons = [
      Person('Henry', 'Singapore'),
      Person('Jane', 'San Francisco'),
      Person('Lee', 'Beijing'),
      Person('John', 'Sydney'),
      Person('Alfred', 'London')
  ]

  (p | beam.Create(persons)
     | beam.ParDo(EnrichCountryDoFn(), beam.pvalue.AsDict(cities_to_countries))
    | 'Write' >> beam.io.textio.WriteToText('example-output'))
    
clear_output()
!cat example-output*

Person[Henry,Singapore,Singapore]
Person[Jane,San Francisco,United States]
Person[Lee,Beijing,China]
Person[John,Sydney,Australia]
Person[Alfred,London,United Kingdom]


# TESTS

In [63]:
%%writefile filtertest.py
#filter test
import logging
import time
import unittest

import pytest
from hamcrest.core.core.allof import all_of


from apache_beam.examples.cookbook import filters
from apache_beam.io.gcp.tests import utils
from apache_beam.io.gcp.tests.bigquery_matcher import BigqueryMatcher
from apache_beam.testing.pipeline_verifiers import PipelineStateMatcher
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to


class FiltersTest(unittest.TestCase):

  # The default checksum is a SHA-1 hash generated from sorted rows reading
  # from expected Bigquery table.
  DEFAULT_CHECKSUM = '813b6da1624334732fad4467d74a7c8a62559c6b'

  # Note that 'removed' should be projected away by the pipeline
  input_data = [
      {
          'year': 2010, 'month': 1, 'day': 1, 'mean_temp': 3, 'removed': 'a'
      },
      {
          'year': 2012, 'month': 1, 'day': 2, 'mean_temp': 3, 'removed': 'a'
      },
      {
          'year': 2011, 'month': 1, 'day': 3, 'mean_temp': 5, 'removed': 'a'
      },
      {
          'year': 2013, 'month': 2, 'day': 1, 'mean_temp': 3, 'removed': 'a'
      },
      {
          'year': 2011, 'month': 3, 'day': 3, 'mean_temp': 5, 'removed': 'a'
      },
  ]

  def _get_result_for_month(self, pipeline, month):
    rows = (pipeline | 'create' >> beam.Create(self.input_data))
    results = filters.filter_cold_days(rows, month)
    return results

  def test_basics(self):
    """Test that the correct result is returned for a simple dataset."""
    with TestPipeline() as p:
      results = self._get_result_for_month(p, 1)
      assert_that(
          results,
          equal_to([{
              'year': 2010, 'month': 1, 'day': 1, 'mean_temp': 3
          }, {
              'year': 2012, 'month': 1, 'day': 2, 'mean_temp': 3
          }]))

  def test_basic_empty(self):
    """Test that the correct empty result is returned for a simple dataset."""
    with TestPipeline() as p:
      results = self._get_result_for_month(p, 3)
      assert_that(results, equal_to([]))

  def test_basic_empty_missing(self):
    """Test that the correct empty result is returned for a missing month."""
    with TestPipeline() as p:
      results = self._get_result_for_month(p, 4)
      assert_that(results, equal_to([]))

  @pytest.mark.examples_postcommit
  def test_filters_output_bigquery_matcher(self):
    test_pipeline = TestPipeline(is_integration_test=True)

    # Set extra options to the pipeline for test purpose
    project = test_pipeline.get_option('project')

    dataset = 'FiltersTestIT'
    table = 'cold_days_%s' % int(round(time.time() * 1000))
    output_table = '.'.join([dataset, table])
    query = 'SELECT year, month, day, mean_temp FROM `%s`' % output_table

    pipeline_verifiers = [
        PipelineStateMatcher(),
        BigqueryMatcher(
            project=project, query=query, checksum=self.DEFAULT_CHECKSUM)
    ]
    extra_opts = {
        'output': output_table,
        'on_success_matcher': all_of(*pipeline_verifiers)
    }

    # Register cleanup before pipeline execution.
    # Note that actual execution happens in reverse order.
    self.addCleanup(utils.delete_bq_table, project, dataset, table)

    # Get pipeline options from command argument: --test-pipeline-options,
    # and start pipeline job by calling pipeline main function.
    filters.run(test_pipeline.get_full_options_as_args(**extra_opts))


if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  unittest.main()

Overwriting filtertest.py


In [64]:
!python3 filtertest.py

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:root:Default Python SDK image for environment is apache/beam_python3.10_sdk:2.48.0
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7f106466e200> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7f106466e320> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function pack_combiners at 0x7f106466e830> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function lift_combiners at 0x7f106466e8c0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_sdf at 0x7f106466ea70> ====================
INFO:apache_beam.runners.portabil

In [65]:




with beam.Pipeline() as p:

    numbers = p | beam.Create([1, 2, 3, 4, 5])

    mult5_results = numbers | beam.Map(lambda num: num * 2)
    mult10_results = numbers | beam.Map(lambda num: num * 10)

    mult5_results | 'Log multiply 5' >> beam.io.textio.WriteToText('example-output')
    mult10_results| 'Log multiply 10' >> beam.io.textio.WriteToText('example-output2')

    
clear_output()
!cat example-output* 


2
4
6
8
10
10
20
30
40
50


reading ~100MB subset for each file | run in terminal | Google Cloud SDK needed
``` bash
gsutil cat -r 0-10000000  gs://apache-beam-samples/game/gaming_data1.csv>game1.csv
gsutil cat -r 0-10000000  gs://apache-beam-samples/game/gaming_data2.csv>game2.csv
head -n -2 game1.csv > game11.csv 
head -n -2 game2.csv > game22.csv
rm game1.csv game2.csv
mv game11.csv game1.csv
mv game22.csv game2.csv
```

In [66]:
%%writefile userscore1.py
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""First in a series of four pipelines that tell a story in a 'gaming' domain.
Concepts: batch processing; reading input from Google Cloud Storage or a from a
local text file, and writing output to a text file; using standalone DoFns; use
of the CombinePerKey transform.
In this gaming scenario, many users play, as members of different teams, over
the course of a day, and their actions are logged for processing. Some of the
logged game events may be late-arriving, if users play on mobile devices and go
transiently offline for a period of time.
This pipeline does batch processing of data collected from gaming events. It
calculates the sum of scores per user, over an entire batch of gaming data
(collected, say, for each day). The batch processing will not include any late
data that arrives after the day's cutoff point.
For a description of the usage and options, use -h or --help.
To specify a different runner:
  --runner YOUR_RUNNER
NOTE: When specifying a different runner, additional runner-specific options
      may have to be passed in as well
EXAMPLES
--------
# DirectRunner
python user_score.py \
    --output /local/path/user_score/output
# DataflowRunner
python user_score.py \
    --output gs://$BUCKET/user_score/output \
    --runner DataflowRunner \
    --project $PROJECT_ID \
    --region $REGION_ID \
    --temp_location gs://$BUCKET/user_score/temp
"""

# pytype: skip-file

# beam-playground:
#   name: UserScore
#   description: batch processing; reading input from Google Cloud Storage or a
#     from a local text file, and writing output to a text file; using
#     standalone DoFns; use of the CombinePerKey transform.
#   multifile: true
#   pipeline_options: --output output.txt
#   context_line: 81
#   categories:
#     - Batch
#     - Combiners
#     - Options
#   complexity: ADVANCED
#   tags:
#     - batch
#     - combine
#     - combine
#     - io
#     - strings

import argparse
import csv
import logging


from apache_beam.metrics.metric import Metrics
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions


class ParseGameEventFn(beam.DoFn):
  """Parses the raw game event info into a Python dictionary.
  Each event line has the following format:
    username,teamname,score,timestamp_in_ms,readable_time
  e.g.:
    user2_AsparagusPig,AsparagusPig,10,1445230923951,2015-11-02 09:09:28.224
  The human-readable time string is not used here.
  """
  def __init__(self):
    # TODO(BEAM-6158): Revert the workaround once we can pickle super() on py3.
    # super().__init__()
    beam.DoFn.__init__(self)
    self.num_parse_errors = Metrics.counter(self.__class__, 'num_parse_errors')

  def process(self, elem):
    try:
      row = list(csv.reader([elem]))[0]
      yield {
          'user': row[0],
          'team': row[1],
          'score': int(row[2]),
          'timestamp': int(row[3]) / 1000.0,
      }
    except:  # pylint: disable=bare-except
      # Log and count parse errors
      self.num_parse_errors.inc()
      logging.error('Parse error on "%s"', elem)


# [START extract_and_sum_score]
class ExtractAndSumScore(beam.PTransform):
  """A transform to extract key/score information and sum the scores.
  The constructor argument `field` determines whether 'team' or 'user' info is
  extracted.
  """
  def __init__(self, field):
    # TODO(BEAM-6158): Revert the workaround once we can pickle super() on py3.
    # super().__init__()
    beam.PTransform.__init__(self)
    self.field = field

  def expand(self, pcoll):
    return (
        pcoll
        | beam.Map(lambda elem: (elem[self.field], elem['score']))
        | beam.CombinePerKey(sum))


# [END extract_and_sum_score]


class UserScore(beam.PTransform):
  def expand(self, pcoll):
    return (
        pcoll
        | 'ParseGameEventFn' >> beam.ParDo(ParseGameEventFn())
        # Extract and sum username/score pairs from the event data.
        | 'ExtractAndSumScore' >> ExtractAndSumScore('user'))


# [START main]
def run(argv=None, save_main_session=True):
  """Main entry point; defines and runs the user_score pipeline."""
  parser = argparse.ArgumentParser()

  # The default maps to two large Google Cloud Storage files (each ~12GB)
  # holding two subsequent day's worth (roughly) of data.
  parser.add_argument(
      '--input',
      type=str,
      # default='gs://apache-beam-samples/game/gaming_data*.csv', # ~24GB total
      default='./game*.csv', # ~200MB total
      
      help='Path to the data file(s) containing game data.')
  parser.add_argument(
      '--output', type=str, required=True, help='Path to the output file(s).')

  args, pipeline_args = parser.parse_known_args(argv)

  options = PipelineOptions(pipeline_args)

  # We use the save_main_session option because one or more DoFn's in this
  # workflow rely on global context (e.g., a module imported at module level).
  options.view_as(SetupOptions).save_main_session = save_main_session

  with beam.Pipeline(options=options) as p:

    def format_user_score_sums(user_score):
      (user, score) = user_score
      return 'user: %s, total_score: %s' % (user, score)

    (  # pylint: disable=expression-not-assigned
        p
        | 'ReadInputText' >> beam.io.ReadFromText(args.input)
        | 'UserScore' >> UserScore()
        | 'FormatUserScoreSums' >> beam.Map(format_user_score_sums)
        | 'WriteUserScoreSums' >> beam.io.WriteToText(args.output))


# [END main]

if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  run()

Overwriting userscore1.py


In [67]:
!python3 userscore1.py --output userscore1-output

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:root:Default Python SDK image for environment is apache/beam_python3.10_sdk:2.48.0
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7f97724cc430> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7f97724cc550> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function pack_combiners at 0x7f97724cca60> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function lift_combiners at 0x7f97724ccaf0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_sdf at 0x7f97724ccca0> ====================
INFO:apache_beam.runners.portabil

In [68]:
!head -n 5 userscore1-output*

user: user16_AmaranthKoala, total_score: 8112
user: user10_AndroidGreenKoala, total_score: 4243
user: user9_AuburnCockatoo, total_score: 3294
user: user1_AntiqueBrassPlatypus, total_score: 7778
user: user9_BattleshipGreyPossum, total_score: 4141


In [69]:
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

--2023-07-14 13:27:31--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58,89K  --.-KB/s    in 0,07s   

2023-07-14 13:27:31 (848 KB/s) - ‘titanic.csv’ saved [60302/60302]



In [152]:
[get_age_group(32) if type("a")==int else  None]

[None]

In [261]:
convert_embarked(None)

''

In [139]:
(get_ticket("4556456"))

'4556456'

In [34]:
!rm -r beam-temp*

rm: cannot remove 'beam-temp*': No such file or directory


In [36]:
!rm example-output*
import csv
def get_age_group(age):
    
    if age < 18:
        return "Child"
    elif age < 35:
        return "Young Adult"
    elif age < 50:
        return "Adult"
    else:
        return None

def get_deck(cabin):
    if cabin:
        return cabin[0]
    else:
        return cabin

def convert_embarked(embarked):
    if embarked == "C":
        return "Cherbourg"
    elif embarked == "Q":
        return "Queenstown"
    elif embarked == "S":
        return "Southampton"
    else:
        return ""

def get_ticket_prefix(ticket):
    if ' ' in ticket:
        return ticket.split(' ')[0].replace('.', '').replace('/', '')
    else:
        return ticket
    
def get_ticket(field):
    try:
        value = re.match(r"\S+\s(\d+)", field).group(1)
        assert type(int(values)) == int
        return value
    except:
        try:
            assert type(int(field)) == int
            return field
        except:
            return 0
    
def process_titanic_data(input_file, output_file):
    with beam.Pipeline() as pipeline:
        # Read the input CSV file
        lines = pipeline | "ReadInputFile" >> beam.io.ReadFromText(input_file)

        # Extract the header
        header, data = (
            lines
            | "SplitHeaderData" >> beam.Partition(lambda line, _: 0 if line.startswith("PassengerId") else 1, 2)
        )

        # Parse the CSV lines into individual fields 
        parsed_data = (
            data
            | "ParseCSV" >> beam.Map(lambda line: next(csv.reader([line])))
        )
        #12
        title_data = (
            parsed_data
            | "AddTitleField" >> beam.Map(lambda fields: fields + [fields[3].split(',')[1].strip().split(' ')[0]])
        )
        #13
        title_data2 = (
            title_data
            | "AddFamilyField" >> beam.Map(lambda fields: fields + [fields[3].split(',')[0].strip()])
        )
        #14
        pclass_age_data = (
            title_data2
            | "AddPclassAgeField" >> beam.Map(lambda fields: fields + [int(fields[2]) * float(fields[5])]
                                              if fields[2] and fields[5] else fields + [""])
        
        )
        
        #15 Add a new field "Family Size" by combining "SibSp" and "Parch"
        family_size_data = (
            pclass_age_data
            | "AddFamilySizeField" >> beam.Map(lambda fields: fields + [int(fields[6]) + int(fields[7]) + 1])
        )
        
        #16 Bin the "Age" field into different age groups
        age_group_data = (
            family_size_data
            | "AddAgeGroupField" >> beam.Map(lambda fields: fields + [get_age_group(float(fields[5]))] if fields[5] else  fields + [""])
        )

        #17 Calculate "Fare Per Person" by dividing "Fare" by family size
        fare_per_person_data = (
            age_group_data
            | "AddFarePerPersonField" >> beam.Map(lambda fields: fields + [int(get_ticket(fields[8]))
                                                                           / (float(fields[9])+0.00001)] if fields[8] 
                                                  and fields[9] else  fields + [""])
        )

        
         # 18 Create a binary feature indicating whether a passenger was traveling alone or not
        is_alone_data = (
            fare_per_person_data
            | "AddIsAloneField" >> beam.Map(lambda fields: fields + [1 if int(fields[6]) + int(fields[7]) == 0 else 0] 
                                            if fields[6] and fields[7] else fields + [""])
        )

        #19 Convert the "Embarked" field into categorical variables
        embarked_data = (
            is_alone_data
            | "ConvertEmbarkedField" >> beam.Map(lambda fields: fields + [convert_embarked(fields[11])] if  fields[11] else  fields + [""])
        )

        #20 Calculate the length of the passenger's name
        name_length_data = (
            embarked_data
            | "AddNameLengthField" >> beam.Map(lambda fields: fields + [len(fields[3])]  if  fields[3] else  fields + [""])
        )

        #21 Extract any prefixes from the "Ticket" field
        ticket_prefix_data = (
            name_length_data
            | "AddTicketPrefixField" >> beam.Map(lambda fields: fields + [get_ticket_prefix(fields[9])]  if  fields[9] else  fields + [""])
        )
        
        
        #22 Extract the deck information from the "Cabin" field
        deck_data = (
            ticket_prefix_data
            | "AddDeckField" >> beam.Map(lambda fields: fields + [get_deck(fields[10])] if  fields[10] else  fields + [""] )
        )
        
        
        
        
        # Convert a specific field from string to int
        converted_data = (
            deck_data
            | "ConvertField" >> beam.Map(lambda fields: (int(fields[0]),
                                                         int(fields[1]),
                                                         int(fields[2]),
                                                         fields[3],
                                                         fields[4],
                                                         fields[5],
                                                         int(fields[6]),
                                                         int(fields[7]),
                                                         fields[8],
                                                         float(fields[9]), 
                                                         fields[10],
                                                         fields[11], 
                                                         fields[12], 
                                                         fields[13], 
                                                         (fields[14]), 
                                                         int(fields[15]), 
                                                         fields[16], 
                                                         float(fields[17]), 
                                                         int(fields[18]),
                                                        fields[19],
                                                        int(fields[20]),
                                                        float(fields[21]),
                                                         fields[22]
                                         
                                        
        )))

        

        # Combine the header and converted data
        combined_data = (
            [converted_data]
            | "CombineData" >> beam.Flatten()
        )
        # Convert the tuples to comma-separated strings
        csv_data = converted_data | "ConvertToCSV" >> beam.Map(lambda row: ",".join(str(value) for value in row))
        
        # Write the processed data to a new CSV file
        csv_data | "WriteToCSV" >> beam.io.WriteToText(output_file, header=None)

if __name__ == '__main__':
    input_file = 'titanic.csv'
    output_file = 'final_example-output-titanic.csv'
    process_titanic_data(input_file, output_file)


rm: cannot remove 'example-output*': No such file or directory


In [37]:
import pandas

In [38]:
df=pd.read_csv("titanic.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [39]:
# cat titanic.csv

In [40]:
cat final_example-output*

1,0,3,Braund, Mr. Owen Harris,male,22,1,0,A/5 21171,7.25,,S,Mr.,Braund,66.0,2,Young Adult,0.0,0,Southampton,23,7.25,
2,1,1,Cumings, Mrs. John Bradley (Florence Briggs Thayer),female,38,1,0,PC 17599,71.2833,C85,C,Mrs.,Cumings,38.0,2,Adult,0.0,0,Cherbourg,51,71.2833,C
3,1,3,Heikkinen, Miss. Laina,female,26,0,0,STON/O2. 3101282,7.925,,S,Miss.,Heikkinen,78.0,1,Young Adult,0.0,1,Southampton,22,7.925,
4,1,1,Futrelle, Mrs. Jacques Heath (Lily May Peel),female,35,1,0,113803,53.1,C123,S,Mrs.,Futrelle,35.0,2,Adult,2143.1822705871427,0,Southampton,44,53.1,C
5,0,3,Allen, Mr. William Henry,male,35,0,0,373450,8.05,,S,Mr.,Allen,105.0,1,Adult,46391.246718948176,1,Southampton,24,8.05,
6,0,3,Moran, Mr. James,male,,0,0,330877,8.4583,,Q,Mr.,Moran,,1,,39118.57096748642,1,Queenstown,16,8.4583,
7,0,1,McCarthy, Mr. Timothy J,male,54,0,0,17463,51.8625,E46,S,Mr.,McCarthy,54.0,1,None,336.7172163476083,1,Southampton,23,51.8625,E
8,0,3,Palsson, Master. Gosta Leonard,male,2,3,1,349909,21.075,,S,Master.,Palsson,6.0,